# Description

This notebook creates timelapse videos of the scattered files in different
folders and organizes them based on the strain of virus and experiment
conditions to be used by trackmate in fiji for tracking.

In [1]:
import os
from pathlib import Path

import tifffile
import numpy as np

In [2]:
PROJECT_ROOT = Path("/Users/ashkanhzdr/workspace/ViralTally")
DATASET_ROOT = PROJECT_ROOT / "dataset" / "Sars2Plaque" / "original"
SAVE_ROOT = PROJECT_ROOT / "dataset" / "Sars2Plaque" / "processed"

In [3]:
def get_ordered_timelapse_file_list(fn, dataset_root=DATASET_ROOT):
    files = []
    for frame_no in range(1, 47):
        dir_name = f"TimePoint_{frame_no}"
        file = dataset_root / dir_name / fn
        if not os.path.exists(file):
            raise FileNotFoundError(f"File not found: {file}")
        files.append(file)
    return files

print(get_ordered_timelapse_file_list(fn="231113-LRRK2i_A01_s1_w1.TIF"))

[PosixPath('/Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/original/TimePoint_1/231113-LRRK2i_A01_s1_w1.TIF'), PosixPath('/Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/original/TimePoint_2/231113-LRRK2i_A01_s1_w1.TIF'), PosixPath('/Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/original/TimePoint_3/231113-LRRK2i_A01_s1_w1.TIF'), PosixPath('/Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/original/TimePoint_4/231113-LRRK2i_A01_s1_w1.TIF'), PosixPath('/Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/original/TimePoint_5/231113-LRRK2i_A01_s1_w1.TIF'), PosixPath('/Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/original/TimePoint_6/231113-LRRK2i_A01_s1_w1.TIF'), PosixPath('/Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/original/TimePoint_7/231113-LRRK2i_A01_s1_w1.TIF'), PosixPath('/Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/original/TimePoint_8/231113-LRRK2i_A01_s1_w1.TIF'), PosixPath('/Users/ashka

In [4]:
def get_1channel_timelapse_from_list(files):
    tiff_data = []
    for f in files:
        tiff_data.append(tifffile.imread(f))
    merged_tiff = np.stack(tiff_data, axis=0)
    return merged_tiff

def merge_timelapse_into_2channel(ch1_tiff, ch2_tiff, save_path=None):
    stacked_data = np.stack([ch1_tiff, ch2_tiff], axis=1)
    if save_path:
        tifffile.imwrite(save_path, stacked_data, imagej=True, metadata={'axes': 'TCYX'})
    return stacked_data

In [5]:
def consolidate_timelapse_data(fn, save_path):
    files1 = get_ordered_timelapse_file_list(fn.replace('channel', 'w1'))
    files2 = get_ordered_timelapse_file_list(fn.replace('channel', 'w2'))
    video1 = get_1channel_timelapse_from_list(files1)
    video2 = get_1channel_timelapse_from_list(files2)
    return merge_timelapse_into_2channel(video1, video2, save_path)

In [6]:
def consolidate_dataset(exp_well_maps, save_root):
    parts = ["s1", "s2", "s3", "s4"]
    for exp_id, wells in exp_well_maps.items():
        os.makedirs(save_root / exp_id, exist_ok=True)
        for well in wells:
            for part in parts:
                save_path = save_root / exp_id / f"{well}_{part}.tiff"
                fn = f"231113-LRRK2i_{well}_{part}_channel.TIF"
                _ = consolidate_timelapse_data(fn=fn, save_path=save_path)
                print(f"Saved {save_path}")

# First set of experiments

In [7]:
wells = {
    "Vero_DMSO": ["A07", "B07", "C07", "D07", "E07", "F07", "G07", "A09", "A10"],
    "Vero_inhibitor": ["A08", "B08", "C08", "D08", "E08", "F08", "G08", "A11", "A12"],
    # "ACE2_A549_DMSO": ["A01", "B01", "C01", "D01", "E01", "F01", "G01", "H01", "A03", "A04"],
    # "ACE2_A549_inhibitor": ["A02", "B02", "C02", "D02", "E02", "F02", "G02", "H02", "A05", "A06"],
}

In [8]:
consolidate_dataset(exp_well_maps=wells, save_root=SAVE_ROOT)

Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/A07_s1.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/A07_s2.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/A07_s3.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/A07_s4.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/B07_s1.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/B07_s2.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/B07_s3.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/B07_s4.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/C07_s1.tiff
Saved /Users/ashkanhzdr/workspace/ViralTally/dataset/Sars2Plaque/processed/Vero_DMSO/C07_s2.tiff
Saved /Users/ashkanhzdr/worksp